In [1]:
import pandas as pd

In [2]:
assessments = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/PBW7aUHGuodFDA')
courses = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/m0Z6QYNT46f9tQ')
studentAssessment = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/lsmdbYB0iM7p3w')
studentRegistration = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Yse4Y6RJqg_WaA')

In [3]:
# Проверим на пустые значения

In [4]:
assessments.isna().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

In [5]:
# date - время в днях от начала презентации до сдачи. Возможно это предметы с необязательными экзаменами?
# Пока нет серьезных оснований предполагать, что это некорректные данные, проанализируем дополнительно после 
# объединения df
# 
# -------------------------- NB1 ------------------------
# 

assessments[assessments.date.isna()]

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0
53,BBB,2014J,15025,Exam,NaN,100.0
62,CCC,2014B,24290,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
72,CCC,2014J,24299,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0


In [6]:
courses.isna().sum()

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64

In [7]:
studentAssessment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

In [8]:
# Сохраним незаполненные оценки (есть смысл в дальнейшем проанализиролвать, насколько отсутсвие данных оценок 
# повлияло на успеваемость студента, возможно это наша ошибка и из-за этого у студента не будет принят 
# экзамен) На текущий момент предполагаем, что это ошибочные данные и удалим их, чтобы не искажали основные 
# расчеты. Наложим эти данные на результаты рассчетов ниже
# 
# -------------------------- NB2 ------------------------
# 

na_scores = studentAssessment[studentAssessment['score'].isna()]
studentAssessment = studentAssessment.dropna(subset=['score'])
studentAssessment['score'] = studentAssessment['score'].astype('int32')

In [9]:
# После объединения df необходимо проанализировать строки с date_registration == NA
# 
# -------------------------- NB3 ------------------------
# 

studentRegistration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [10]:
# Проверим указанные в задании условия.
# На каждом предмете есть несколько презентаций (семестров), по каждой презентации есть оценки.
# Сумма их весов равна 100
# На GGG weight всех промежуточных оценок = 0

assessments[assessments['assessment_type'] != 'Exam'] \
    .groupby(['code_module', 'code_presentation']).agg({'weight': 'sum'})

weight
code_module code_presentation        
AAA         2013J               100.0
            2014J               100.0
BBB         2013B               100.0
            2013J               100.0
            2014B               100.0
            2014J               100.0
CCC         2014B               100.0
            2014J               100.0
DDD         2013B               100.0
            2013J               100.0
            2014B               100.0
            2014J               100.0
EEE         2013J               100.0
            2014B               100.0
            2014J               100.0
FFF         2013B               100.0
            2013J               100.0
            2014B               100.0
            2014J               100.0
GGG         2013J                 0.0
            2014B                 0.0
            2014J                 0.0

In [11]:
# На предмете GGG при этом веса для экзаменов установлены.
# Возможно это ошибка в данных или для данного предмета требование промежуточных результатов не актуально?
# Необходим отдельный анализ для данного предмета после объединения df.
# 
# -------------------------- NB4 ------------------------
# 

assessments[assessments['code_module'] == 'GGG']

,code_module,code_presentation,id_assessment,assessment_type,date,weight
176,GGG,2013J,37418,CMA,229.0,0.0
177,GGG,2013J,37419,CMA,229.0,0.0
178,GGG,2013J,37420,CMA,229.0,0.0
179,GGG,2013J,37421,CMA,229.0,0.0
180,GGG,2013J,37422,CMA,229.0,0.0
181,GGG,2013J,37423,CMA,229.0,0.0
182,GGG,2013J,37415,TMA,61.0,0.0
183,GGG,2013J,37416,TMA,124.0,0.0
184,GGG,2013J,37417,TMA,173.0,0.0
185,GGG,2013J,37424,Exam,229.0,100.0


In [12]:
# Посмотрим на число экзаменов по предметам и семестрам
# По каждой презентации помимо тестов есть экзамены, обычно 1
# Вес каждого экзамена действительно 100
# 
# -------------------------- NB5 ------------------------
# 

assessments[assessments['assessment_type'] == 'Exam'] \
    .groupby(['code_module', 'code_presentation']) \
    .agg({'weight': ['count', 'sum']})

weight       
                               count    sum
code_module code_presentation              
AAA         2013J                  1  100.0
            2014J                  1  100.0
BBB         2013B                  1  100.0
            2013J                  1  100.0
            2014B                  1  100.0
            2014J                  1  100.0
CCC         2014B                  2  200.0
            2014J                  2  200.0
DDD         2013B                  1  100.0
            2013J                  1  100.0
            2014B                  1  100.0
            2014J                  1  100.0
EEE         2013J                  1  100.0
            2014B                  1  100.0
            2014J                  1  100.0
FFF         2013B                  1  100.0
            2013J                  1  100.0
            2014B                  1  100.0
            2014J                  1  100.0
GGG         2013J                  1  100.0
            2014B                  1  100.0
            2014J                  1  100.0

In [13]:
# На курсе ССС в каждом семестре по 2 экзамена, посмотрим поближе

CCC_exams = assessments \
    .query('(code_module == "CCC") & (assessment_type == "Exam")') \
    .id_assessment \
    .tolist()
CCC_exams

[24290, 40087, 24299, 40088]

In [14]:
# По id_assessment 40087 и 40088 оценок нет, следовательно не обращаем внимания на то, что
# на курсе CCC по 2 экзамена на семестр
# на этапе объединения (следующая ячейка) они не будут включены в рабочий df

studentAssessment \
    .query('id_assessment in @CCC_exams') \
    .id_assessment \
    .value_counts()

24299    1168
24290     747
Name: id_assessment, dtype: int64

In [15]:
# Подготовим df для работы над первыми заданиями:
# 1. assessments inner join с оценками студентов, т.к. если id_assessment в studentAssessment отсутствует, то 
# данные не валидны или не репрезантивны с точки зрения задания. В исходном df они останутся для решения 
# других возможных задач.
# 2. left join studentRegistration, чтобы добавить сведения об отмене регистрации

df_a = assessments \
    .merge(studentAssessment, how='inner', on='id_assessment') \
    .merge(studentRegistration, how='left', on=['code_module', 'code_presentation', 'id_student']) \
    .merge(courses, on=['code_module', 'code_presentation'])

Проверим некоторые тезисы NB, обозначенные выше

In [16]:
# NB1
# Новые данные также пока не позволяют сделать однозначных выводов относительно пустых date (срок сдачи экзамена)
# Записей довольно много, больше половины от общего числа записей по экзаменам, причем все относятся к
# семестрам 2014 года, возможно это еще не завершенные уроки
# Пока оставляем NA значения, что на текущий момент ограничивает возможности по использованию данного поля

print(df_a[df_a.date.isna()].shape[0])
print(df_a.query('assessment_type == "Exam"').shape[0])

2865
4959


In [17]:
# NB3 - начало
# Пустые значения в date_registration

df_a[df_a.date_registration.isna()]

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length
46922,CCC,2014B,24286,CMA,18.0,2.0,575419,20,0,0,NaN,-24.0,241
76867,DDD,2013J,25348,TMA,25.0,10.0,550142,-1,1,64,NaN,-196.0,261
77464,DDD,2013J,25349,TMA,53.0,12.5,550142,-1,1,68,NaN,-196.0,261
78881,DDD,2013J,25350,TMA,88.0,17.5,550142,-1,1,82,NaN,-196.0,261
80528,DDD,2013J,25351,TMA,123.0,20.0,550142,-1,1,16,NaN,-196.0,261
89457,DDD,2014J,25363,TMA,41.0,10.0,604632,41,0,85,NaN,12.0,262
96399,EEE,2013J,30709,TMA,33.0,16.0,2681277,32,0,94,NaN,-166.0,268


In [18]:
# Всего 4 студента, ни одного со сдачей экзамена, причем везде, где нет даты регистрации
# есть отмена регистрации, поэтому удаляем строки с незаполненной датой регистрации

na_reg = df_a[df_a.date_registration.isna()].id_student.unique()
df_a.query('id_student in @na_reg')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length
46922,CCC,2014B,24286,CMA,18.0,2.0,575419,20,0,0,NaN,-24.0,241
65918,DDD,2013B,25341,CMA,23.0,2.0,550142,25,0,50,-58.0,212.0,240
66867,DDD,2013B,25342,CMA,51.0,3.0,550142,54,0,58,-58.0,212.0,240
67539,DDD,2013B,25343,CMA,79.0,3.0,550142,81,0,44,-58.0,212.0,240
68440,DDD,2013B,25344,CMA,114.0,4.0,550142,117,0,40,-58.0,212.0,240
69132,DDD,2013B,25345,CMA,149.0,4.0,550142,152,0,61,-58.0,212.0,240
70984,DDD,2013B,25334,TMA,25.0,7.5,550142,24,0,64,-58.0,212.0,240
72044,DDD,2013B,25335,TMA,53.0,10.0,550142,60,0,68,-58.0,212.0,240
72880,DDD,2013B,25336,TMA,81.0,12.5,550142,88,0,82,-58.0,212.0,240
73468,DDD,2013B,25337,TMA,116.0,15.0,550142,130,0,16,-58.0,212.0,240


In [19]:
# NB3 - конец
# удаляем строки с незаполненной датой регистрации

df_a = df_a.dropna(subset=['date_registration'])

Задание 1
Сколько студентов успешно сдали только один курс? 
Успешная сдача — studentAssessment.score для assessment_type == Exam >= 40.

Необходимо обозначить понятие "сдать курс", ниже приведено обоснование, почему классическая модель (несколько семестров, а затем экзамен) не подходит.

Так как "заключительные экзамены не принимаются, если результат предварительных тестов отсутствуют в системе", то успешная сдача курса означает получение внутри семестра суммы весов оценок за тесты (кроме экзаменов) равной 100 и получение оценки за экзамен в этом семестре (потоке) не ниже 40.

В условиях задачи не сказано, что дата сдачи теста или экзамена как-то влияет на прием оценки, поэтому считаем, что тест или экзамен сданный позднее установленного срока все равно принимается.

Успешная сдача — это зачёт по курсу на экзамене.
Посмотрим на сдачу экзаменов по курсам и семестрам.

In [20]:
# Экзамены сдавались только на курсах CCC и DDD, что отсеивает около половины данных. Данный аспект 
# обязательно подсвечиваем продакт менеджеру. Так как остальные курсы не имеют экзаменов, то и критерии 
# оценки успешности их сдачи другиеб нам неизвестные. 
# Сейчас продолжаем работать в рамках поставленной задачи, то есть показатель успешного окончания курса - 
# сдача экзамена при условии сдачи всех промежуточных тестов и получение оцценки за экзамен не ниже 40. 

df_a \
    .query('assessment_type == "Exam"') \
    .groupby(['code_module', 'code_presentation']) \
    .agg({'id_student': 'count'})

id_student
code_module code_presentation            
CCC         2014B                     747
            2014J                    1168
DDD         2013B                     602
            2013J                     968
            2014B                     524
            2014J                     950

In [21]:
# Так как сейчас анализируем завершенные уроки, то оставляем только данные, где date_unregistration не заполнена

df_active = df_a.loc[df_a['date_unregistration'].isna(), :]
df_active

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78,-159.0,NaN,268
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70,-53.0,NaN,268
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72,-52.0,NaN,268
3,AAA,2013J,1752,TMA,19.0,10.0,32885,26,0,69,-176.0,NaN,268
4,AAA,2013J,1752,TMA,19.0,10.0,38053,19,0,79,-110.0,NaN,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173734,GGG,2014J,37437,TMA,173.0,0.0,652462,172,0,60,-115.0,NaN,269
173735,GGG,2014J,37437,TMA,173.0,0.0,652539,176,0,75,-85.0,NaN,269
173736,GGG,2014J,37437,TMA,173.0,0.0,653157,187,0,70,-46.0,NaN,269
173737,GGG,2014J,37437,TMA,173.0,0.0,653252,171,0,70,-78.0,NaN,269


In [22]:
# Обоснование неприменимости модели "несколько семестров, а затем экзамен"
# Необходимо принять решение о значении термина: code_presentation == семестр == поток
# Обоснование: существуют единицы студентов, которые присутсвовали более чем в одном семестре одного модуля
# и при этом сдавали экзамен (то есть классическая модель: несколько семестров, а затем экзамен - не подходит)
# ну или у нас очень плохой продукт, что крайне маловероятно, пропускаем.

students_2_pres = df_active \
    .groupby(['id_student', 'code_module'], as_index=False) \
    .agg({'code_presentation': 'nunique'}) \
    .query('code_presentation > 1') \
    .id_student \
    .tolist()
df_active.query('id_student in @students_2_pres') \
    .groupby(['id_student', 'code_module', 'code_presentation', 'assessment_type'], as_index=False) \
    .agg({'weight': 'sum'}) \
    .query('assessment_type == "Exam"')

,id_student,code_module,code_presentation,assessment_type,weight
1,111926,DDD,2014J,Exam,100.0
42,416008,DDD,2014B,Exam,100.0
59,465629,DDD,2014J,Exam,100.0
94,527323,DDD,2013B,Exam,100.0
96,527323,DDD,2014B,Exam,100.0
107,540758,DDD,2013B,Exam,100.0
109,540758,DDD,2014B,Exam,100.0
133,552114,DDD,2014J,Exam,100.0
148,555408,DDD,2013B,Exam,100.0
152,556370,DDD,2013B,Exam,100.0


In [23]:
# 243 записи по студентам, присутсвующим более чем в одном семестре (без ограничения на сдачу экзамена), значит
# действительно принимаем, что закончить курс == закончить семестр.

df_active.query('id_student in @students_2_pres') \
    .groupby(['id_student', 'code_module', 'code_presentation', 'assessment_type'], as_index=False) \
    .agg({'weight': 'sum'})

,id_student,code_module,code_presentation,assessment_type,weight
0,111926,DDD,2013J,TMA,40.0
1,111926,DDD,2014J,Exam,100.0
2,111926,DDD,2014J,TMA,65.0
3,142324,BBB,2013B,CMA,3.0
4,142324,BBB,2013B,TMA,59.0
...,...,...,...,...,...
238,2400764,DDD,2014J,TMA,75.0
239,2697921,FFF,2013J,CMA,0.0
240,2697921,FFF,2013J,TMA,100.0
241,2697921,FFF,2014J,CMA,0.0


In [24]:
# Проверим дубли и продолжаем

df_active[df_active.duplicated()]

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length


План: 
1. Работаем только с не отменившими регистрацию (df_active)
2. Соберем датафрейм, в котором сохраним данные о студентах, допущенных к сдаче экзамена по курсу (потоку).
2.1. Отберем assessment_type != Exam
2.2. Группируем по студенту, предмету, потоку (семестру) и считаем сумму весов оценок, оставим только равные 100
3. Попутно проверим, не записываются ли данные о повторных прохождениях экзаменов
4. Пересечем получившийся df с данными об оценках за экзамен по ключу студент+курс+семестр, добавив сведения об оценке
5. Оставим только студентов с оценкой за экзамен не ниже 40

In [25]:
# Студенты, допущенные к сдаче экзамена по курсу (потоку)

admitted_students = df_active \
    .query('assessment_type != "Exam"') \
    .groupby(['id_student', 'code_module', 'code_presentation'], as_index=False) \
    .agg({'weight': 'sum'}) \
    .query('weight == 100') \
    .drop(columns=['weight'])

In [26]:
# Дополнительно добавим флаг, обозначающий возможность приема оценки за экзамен в указанном семестре курса
# Понадобится в дальнейших заданиях

admitted_students['admitted'] = True
admitted_students.head()

,id_student,code_module,code_presentation,admitted
0,6516,AAA,2014J,True
1,11391,AAA,2013J,True
3,23698,CCC,2014J,True
4,23798,BBB,2013J,True
6,24213,DDD,2014B,True


In [27]:
# 3. Попутно проверим, не записываются ли данные о повторных прохождениях экзаменов
# Нет, только 1 запись с weight == 100 на 1 экзамен, NA также нет (проверено отдельно, честно :) )

df_active \
    .query('assessment_type == "Exam"') \
    .groupby(['id_student', 'code_module', 'code_presentation']) \
    .agg({'weight': 'sum'}) \
    .describe()

,weight
count,4958.0
mean,100.0
std,0.0
min,100.0
25%,100.0
50%,100.0
75%,100.0
max,100.0


In [28]:
# exam_scores - запишем оценки за экзамены

exam_scores = df_active[['id_student', 'code_module', 'code_presentation',
                         'assessment_type', 'id_assessment', 'score']] \
    .query('assessment_type == "Exam"') \
    .drop(columns=['assessment_type'])

# 4. Пересечем получившийся df с данными об оценках за экзамен
# 5. Оставим только студентов с оценкой за экзамен не ниже 40
# 
# Получим ответ: 2796 студентов успешно сдали только 1 курс

admitted_students.merge(exam_scores, on=['id_student', 'code_module', 'code_presentation']) \
    .query('score >= 40') \
    .groupby('id_student') \
    .agg({'code_module': 'nunique'}) \
    .value_counts()

code_module
1              2796
2               178
dtype: int64

In [29]:
# Запишем ответ в отдельную переменную 
# (названия колонок не переименовываем, так как отсутвует необходимость дальнейшего их использования.
# Это допустимо? Или хорошей практикой будет ВСЕГДА переименовывать имена колонок?)

passed_one_module = admitted_students \
    .merge(exam_scores, on=['id_student', 'code_module', 'code_presentation'], how='left') \
    .query('score >= 40') \
    .groupby('id_student') \
    .agg({'code_module': 'nunique'}) \
    .query('code_module == 1') \
    .count()[0]
passed_one_module

2796

Задание 2. 
Выяви самый сложный и самый простой экзамены: найди курсы и экзамены в рамках курса, 
которые обладают самой низкой и самой высокой завершаемостью

завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

Есть студент, который через 5 дней после отмены регистрации сдал экзамен. Такого быть не должно, возможно ошибка в настройках площадки: например доступ к экзамену остался после отмены регистрации, - подсветить этот кейс. Если действительно проблема с доступом, то студент, отменивший регистрацию мог неполностью выполнить задачи экзамена (зашел из интереса посмотреть варианты) и доверять его оценке не стоит, следовательно включать этот результат в общее число попыток также не следует. Поэтому продолжаем рассматривать в рамках задачи df_active - сведения, включающие студентов, не отменивших регистрацию.

In [30]:
df_a[~df_a.date_unregistration.isna()].query('assessment_type == "Exam"')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length
83108,DDD,2013J,25354,Exam,261.0,100.0,425543,237,0,33,-39.0,232.0,261


Добавим мектрику завершаемости GR - graduation rate, которую определим как отношение числа экзаменов с оценкой не ниже 40 у студентов, допущенных к экзамену (сумма weight в данном семестре промежуточных тестов = 100, admitted_students из задания 1) к общему числу записей с оценками по данному экзамену В задании 1 (п.3) мы выявили, что пересдачи экзаменов нет, то есть у 1 студента есть только 1 запись на 1 экзамен, проверим:

In [31]:
df_active \
    .query('assessment_type == "Exam"') \
    .groupby(['id_student', 'code_module', 'code_presentation']) \
    .agg({'score': 'count'}) \
    .value_counts()

score
1        4958
dtype: int64

План
1. Для каждой записи типа "экзамен" посчтиаем GR
2. Отсортируем по убыванию GR, выведем в ответ абсолютные min и max по метрике
3. Объединим через inner join полученный df с оценками по экзаменам (assessments), сгруппируем по курсу и выведем min и max по каждому

In [32]:
# Число успешно сданных экзаменов
# Используем подготовленные в 1 задании df: 
# admitted_students - студенты, допущенные к экзамену, соединенные с
# exam_scores - оценки за экзамены, отобранные по условию "оценка не ниже 40"

exam_passes = admitted_students.merge(exam_scores, on=['id_student', 'code_module', 
                                                       'code_presentation'], how='left') \
    .query('score >= 40')

exam_passes['id_assessment'] = exam_passes['id_assessment'].astype('int')
exam_passes

,id_student,code_module,code_presentation,admitted,id_assessment,score
2,23698,CCC,2014J,True,24299,80.0
4,24213,DDD,2014B,True,25361,58.0
11,27116,CCC,2014J,True,24299,96.0
15,28046,DDD,2013J,True,25354,40.0
20,29411,DDD,2013J,True,25354,62.0
...,...,...,...,...,...,...
12455,2693243,DDD,2013B,True,25340,82.0
12457,2694886,DDD,2014B,True,25361,69.0
12459,2694933,DDD,2013B,True,25340,73.0
12460,2695608,DDD,2013J,True,25354,73.0


In [33]:
# Посчитаем число успешно сданных экзаменов

exam_passes = exam_passes \
    .groupby('id_assessment') \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'pass_count'})
exam_passes

,pass_count
id_assessment,
24290,393
24299,642
25340,317
25354,684
25361,368
25368,750


In [34]:
# Число попыток сдачи экзаменов

exam_attempts = df_active \
    .query('assessment_type == "Exam"') \
    .groupby('id_assessment') \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'attempt_count'})
exam_attempts

,attempt_count
id_assessment,
24290,747
24299,1168
25340,602
25354,967
25361,524
25368,950


In [35]:
exams = exam_passes.merge(exam_attempts, left_index=True, right_index=True)
exams

,pass_count,attempt_count
id_assessment,,
24290,393,747
24299,642,1168
25340,317,602
25354,684,967
25361,368,524
25368,750,950


In [36]:
exams['GR'] = exams['pass_count'] / exams['attempt_count']
exams.sort_values('GR')

,pass_count,attempt_count,GR
id_assessment,,,
24290,393,747,0.526104
25340,317,602,0.526578
24299,642,1168,0.549658
25361,368,524,0.702290
25354,684,967,0.707342
25368,750,950,0.789474


In [37]:
# Самый легкий экзамен среди всех курсов, завершаемость около 79%

exams.GR.idxmax()

25368

In [38]:
# Самый сложный экзамен среди всех курсов, завершаемость около 53%

exams.GR.idxmin()

24290

In [39]:
# Теперь выведем самый легкий и самый сложный экзамен в рамках каждого курса, переведем значения метрики в %
# hardest_exam - id самого сложного экзамена, GR_hardest - его завершаемость в %
# easiest_exam - id самого легкого экзамена, GR_easiest - его завершаемость в %


exam_difficulty = assessments \
    .merge(exams, on='id_assessment') \
    .set_index('id_assessment') \
    .groupby('code_module') \
    .agg({'GR': ['idxmin', 'min', 'idxmax', 'max']})

exam_difficulty.columns = exam_difficulty.columns.get_level_values(1)

exam_difficulty = exam_difficulty.rename(columns={'idxmin': 'hardest_exam', 'min': 'GR_hardest',
                                                  'idxmax': 'easiest_exam', 'max': 'GR_easiest'})

exam_difficulty['GR_hardest'] = round(exam_difficulty['GR_hardest'] * 100, 1)
exam_difficulty['GR_easiest'] = round(exam_difficulty['GR_easiest'] * 100, 1)

exam_difficulty

,hardest_exam,GR_hardest,easiest_exam,GR_easiest
code_module,,,,
CCC,24290,52.6,24299,55.0
DDD,25340,52.7,25368,78.9


Есть предположение, что продакт-менеджеру будут не информативны ID экзаменов
Вывести название семестра не проблема, просто в ячейке выше при необходимости поменяем id_assessment на code_presentation Возможно будут интересны и id, и названия семестров (добавил реализацию этого варианта для интереса)

In [40]:
exam_difficulty = assessments \
    .merge(exams, on='id_assessment') \
    .set_index(['id_assessment', 'code_presentation']) \
    .groupby('code_module') \
    .agg({'GR': ['idxmin', 'min', 'idxmax', 'max']})

exam_difficulty.columns = exam_difficulty.columns.get_level_values(1)

exam_difficulty = exam_difficulty.rename(columns={'idxmin': 'hardest_exam', 'min': 'GR_hardest',
                                                  'idxmax': 'easiest_exam', 'max': 'GR_easiest'})

exam_difficulty['GR_hardest'] = round(exam_difficulty['GR_hardest'] * 100, 1)
exam_difficulty['GR_easiest'] = round(exam_difficulty['GR_easiest'] * 100, 1)

exam_difficulty['hardest_exam'] = exam_difficulty['hardest_exam'].apply(list)

exam_difficulty[['hardest_exam_id', 'hardest_exam_semester']] = \
    pd.DataFrame(exam_difficulty['hardest_exam'].tolist(), index=exam_difficulty.index)
    
exam_difficulty[['easiest_exam_id', 'easiest_exam_semester']] = \
    pd.DataFrame(exam_difficulty['easiest_exam'].tolist(), index=exam_difficulty.index)

exam_difficulty = exam_difficulty[['hardest_exam_id', 'hardest_exam_semester', 'GR_hardest',
                                   'easiest_exam_id', 'easiest_exam_semester', 'GR_easiest']]

exam_difficulty

,hardest_exam_id,hardest_exam_semester,GR_hardest,easiest_exam_id,easiest_exam_semester,GR_easiest
code_module,,,,,,
CCC,24290,2014B,52.6,24299,2014J,55.0
DDD,25340,2013B,52.7,25368,2014J,78.9


Задание 3. 
По каждому предмету определи средний срок сдачи экзаменов 
Под сдачей понимаем последнее успешное прохождение экзамена студентом.

План.
Все также смотрим на df admitted_students, где записаны id студентов, допущенных к экзаменам
1. Добавим сведения о дате сдачи экзамена студентом (персечение по ключам 'code_module', 'code_presentation', 'id_student')
2. Сгруппируем по предмету, сагрегируем среднее время сдачи экзамена студентом

In [41]:
admitted_students \
    .merge(df_active.query('assessment_type == "Exam"')[['code_module', 'code_presentation',
                                                         'id_student', 'date_submitted']],
           on=['code_module', 'code_presentation', 'id_student']) \
    .groupby('code_module') \
    .agg({'date_submitted': 'mean'}) \
    .round(1) \
    .rename(columns={'date_submitted': 'pass_mean'})

,pass_mean
code_module,
CCC,239.7
DDD,238.5


Задание 4. 
Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3).

In [42]:
# Студенты без даты регистрации как правило имеют дату отмены регистрации, число date_registration и 
# date_unregistration на 1-2 порядка больше, чем число записей с пустой датой регистрации, оставляем.

studentRegistration.date_registration.count() + studentRegistration.date_unregistration.count()

42620

In [43]:
studentRegistration.date_registration.isna().sum()

45

План
Здесь будет достаточно исходной таблицы studentRegistration
У студентов есть повторные регистрации, нет причин их убирать, оставляем
1. Будем работать с count, поэтому NA нам не помешают. 
2. Группируем по предмету, аггрегируем количество non-NA записей по дате регистрации, сортируем по убыванию, забираем head(3)
3. Группируем по предмету, аггрегируем количество non-NA записей по дате отмены регистрации, сортируем по убыванию, забираем head(3)

In [44]:
# число записей с пустой датой регистрации

studentRegistration[studentRegistration.date_registration.isna()].id_student.count()

45

In [45]:
# Число регистраций и отмен регистраций по предметам для оценки порядка чисел.

module_reg = studentRegistration \
    .groupby('code_module') \
    .agg({'date_registration': 'count', 'date_unregistration': 'count'})
module_reg

,date_registration,date_unregistration
code_module,,
AAA,748,126
BBB,7900,2377
CCC,4426,1947
DDD,6257,2235
EEE,2932,718
FFF,7751,2380
GGG,2534,289


In [46]:
# Самые популярные курсы (ТОП-3)

registration_top = studentRegistration \
    .groupby('code_module') \
    .agg({'date_registration': 'count'}) \
    .sort_values('date_registration', ascending=False) \
    .head(3)
registration_top

,date_registration
code_module,
BBB,7900
FFF,7751
DDD,6257


In [47]:
# Курсы с самым большим оттоком (ТОП-3)
# Как некрасиво получилось, самые популярные предметы ожидаемо имеют самый высокий абослютный уровень оттока
# Тем не менее эти данные тоже подсветим, могут быть интересны.

outflow_abs_top = studentRegistration \
    .groupby('code_module') \
    .agg({'date_unregistration': 'count'}) \
    .sort_values('date_unregistration', ascending=False) \
    .head(3)
outflow_abs_top

,date_unregistration
code_module,
FFF,2380
BBB,2377
DDD,2235


In [48]:
# Переведем отток в относительную шкалу, пронормируем число отмен регистраций на общее число регистраций по 
# курсу и уже полученный показатель (который сразу для Василия переведем в проценты) применим для сортировки

module_reg['outflow'] = round((module_reg['date_unregistration'] / module_reg['date_registration']) * 100, 1)
module_reg.sort_values('outflow', ascending=False)

,date_registration,date_unregistration,outflow
code_module,,,
CCC,4426,1947,44.0
DDD,6257,2235,35.7
FFF,7751,2380,30.7
BBB,7900,2377,30.1
EEE,2932,718,24.5
AAA,748,126,16.8
GGG,2534,289,11.4


In [49]:
# На выход дадим запрошенные топ-3 предмета по оттоку, но сделаем оговорку, что разница между топ-3 
# и топ-4 около 0,6%. Обязательно делаем акцент, что это относительный уровень оттока
# *удаление NA значений в датах регистрации не влияет на топ-3 список и его порядок

outflow_rel_top = module_reg[['outflow']].sort_values('outflow', ascending=False).head(3)
outflow_rel_top

,outflow
code_module,
CCC,44.0
DDD,35.7
FFF,30.7


Задание 5.
Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. 
В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.

1. Признак формирования когорты: регистрация на семестр с пустой датой отмены регистрации
2. Размер когорты — временной интервал: все регситрации на семестр (есть студенты, совершившие регистрацию как до, так и после начала семестра).
3. Отчетный период — время исследования поведения групп: с начала 2013 по конец 2014 года.
4. Анализируемый ключевой показатель: завершаемость семестра и средний срок сдачи курса

План
1. Когорта == студенты, зарегистрировавшиеся на семестр курса
2. Напишем функцию, которая на вход будет принимать исходный рабочий датафрейм, анализируемый ключевой показатель, период начала и период окончания отчетного периода и возвращать оформленный датафрейм с выбранной метрикой по когортам
3. Необходимо сделать оговорку о датах. В данных представлены только относительные измерения дат (число дней от события), продолжительность семестров больше чем половина года, значит семестры с постфиксом J, B пересекаются и отсюда получить период не удастся. Однако делаем предположение, что название семестра code_presentation в первых 4 символах содержит год, тогда будем брать эти данные для фильтрации. Понятно, что всего два года в данных и представлено, но надеемся, что дела пойдут в гору и данных нальется побольше :)
4. В начало работы функции добавим небольшую обработку некоторых ошибок

In [50]:
def cohorot_analysis(df_cohort, mode='gr_calc', period_start=2013, period_end=2014):
    
    # Начало провеок на ошибки ввода параметров
    try:
        period_start = int(period_start)
    except ValueError:
        return 'Error in "period_start" value. Expected year in int(YYYY) format.'

    try:
        period_end = int(period_end)
    except ValueError:
        return 'Error in "period_end" value. Expected year in int(YYYY) format.'
    
    if period_start > period_end:
        return '"period_start" value not less or equal to "period_end"'
    
    min_year = df_cohort.assign(year=lambda x: x.code_presentation.str[:4].astype('int')).year.min()
    max_year = df_cohort.assign(year=lambda x: x.code_presentation.str[:4].astype('int')).year.max()
    
    if period_start < min_year or period_start < 1900 or period_start > max_year:
        period_start = min_year
        print('Setting "period_start" parameter to minimum year in data')
    
    if period_end > max_year or period_end > 2200 or period_end < min_year:
        period_end = max_year
        print('Setting "period_end" parameter to maximum year in data')
    # Конец проверок на ошибки
    
    # Приведенный ниже расчет подробно не комментируется, так как практически полностью повторяет код, написанный выше
    # для решения других заданий, собранный в одну строку.
    
    # Завершаемость по когортам (семестрам)
    # Проверяем на допуск к экзамену студента (admittet_students), отбираем результаты экзаменов не меньше 40,
    # считаем число успешных экзаменов и число попыток по каждому потоку (курс+семестр), рассчитываем GR для
    # потока, вращаем датафрейм (pivot), применяем форматирование.
    if mode == 'gr_calc':
        return df_cohort \
            .assign(year=lambda x: x.code_presentation.str[:4].astype('int')) \
            .query('(assessment_type != "Exam") & (year >= @period_start) & (year <= @period_end)') \
            .groupby(['id_student', 'code_module', 'code_presentation'], as_index=False) \
            .agg({'weight': 'sum'}) \
            .query('weight == 100') \
            .drop(columns=['weight']) \
            .merge(df_cohort[['id_student', 'code_module', 'code_presentation', 'assessment_type', 'id_assessment',
                              'score']].query('assessment_type == "Exam"').drop(columns=['assessment_type']),
                   on=['id_student', 'code_module', 'code_presentation']) \
            .query('score >= 40') \
            .groupby(['code_module', 'code_presentation']) \
            .agg({'id_student': 'count'}) \
            .rename(columns={'id_student': 'pass_count'}) \
            .merge(df_cohort
                   .query('assessment_type == "Exam"')
                   .groupby(['code_module', 'code_presentation'])
                   .agg({'id_student': 'count'})
                   .rename(columns={'id_student': 'attempt_count'}),
                   left_index=True, right_index=True) \
            .assign(GR=lambda x: x.pass_count / x.attempt_count) \
            .drop(columns=['pass_count', 'attempt_count']) \
            .reset_index() \
            .pivot(index='code_module', columns='code_presentation', values='GR') \
            .style \
            .set_caption(f'Graduation rate by cohort (semester) from {period_start} to {period_end}') \
            .background_gradient(cmap='plasma') \
            .highlight_null('white') \
            .format("{:.2%}", na_rep="") \
            .highlight_min(axis=1, color='red')
        
    # Средние сроки сдачи экзаменов по когортам (семестрам)
    # Аналогично расчету среднего времени сдачи курсов из задания 3.
    elif mode == 'avg_calc':
        return df_cohort \
            .assign(year=lambda x: x.code_presentation.str[:4].astype('int')) \
            .query('(assessment_type != "Exam") & (year >= @period_start) & (year <= @period_end)') \
            .groupby(['id_student', 'code_module', 'code_presentation'], as_index=False) \
            .agg({'weight': 'sum'}) \
            .query('weight == 100') \
            .drop(columns=['weight']) \
            .merge(df_cohort.query('assessment_type == "Exam"')[['code_module', 'code_presentation', 
                                                                 'id_student', 'date_submitted']],
                   on=['code_module', 'code_presentation', 'id_student']) \
            .groupby(['code_module', 'code_presentation']) \
            .agg({'date_submitted': 'mean'}) \
            .round(1) \
            .rename(columns={'date_submitted': 'pass_mean'}) \
            .reset_index() \
            .pivot(index='code_module', columns='code_presentation', values='pass_mean') \
            .style \
            .set_caption(f'Average graduation period by cohort (semester) from {period_start} to {period_end}') \
            .background_gradient(cmap='plasma') \
            .highlight_null('white') \
            .format("{}", na_rep="") \
            .highlight_max(axis=1, color='red')
    else:
        return 'Error in "mode" value. Expected one of: "gr_calc", "avg_calc".'

In [51]:
cohorot_analysis(df_active, mode='gr_calc', period_start=2013, period_end=2014)

code_presentation,2013B,2013J,2014B,2014J
code_module,,,,
CCC,,,52.61%,54.97%
DDD,52.66%,70.73%,70.23%,78.95%


In [52]:
cohorot_analysis(df_active, mode='avg_calc', period_start=2013, period_end=2014)

code_presentation,2013B,2013J,2014B,2014J
code_module,,,,
CCC,,,231.7,244.5
DDD,230.3,239.6,234.9,242.8


Задание 6.
Построй адаптированные под задачу обучения RFM-кластеры студентов. Где:
R — среднее время сдачи одного экзамена, 
F — завершаемость курсов, 
M — среднее количество баллов, получаемое за экзамен. 
Подробно опиши, как ты создавал кластеры.

R — среднее время сдачи одного экзамена - разобьем на 3 подгруппы:
1. Не укладываются в сроки - сдавшие экзамен после дедлайна либо не добрался до экзамена
2. Средняя скорость сдачи - срок сдачи выше среднего, но ниже дедлайна
3. Высокая скорость сдачи - срок сдачи ниже среднего (а значит быстрее)

F — завершаемость курсов - разобьем на 3 подгруппы:
1. Не завершают курсы - завершили 0 курсов
2. Завершили часть курсов - завершили хотя бы 1 курс
3. Завершили все курсы - завершили все курсы, на которые зарегистрировались

M — среднее количество баллов, получаемое за экзамен:
1. Низкие оценки - сдают экзамен с оценкой ниже 40 или не сдавали экзамен
2. Средние оценки - получают оценки в диапазоне от 40 до 74
3. Высокие оценки - оценка 75 и выше

Нам понадобятся полные данные (df_a), так как студенты с 0-й завершаемостью часто отменяют регистрацию.
При расчете F для каждого студента необходимо рассчитать его GRS (Graduation Rate by Student) - завершаемость курсов по студенту. GR, рассчитываемая во 2 задании не подходит, так как относится к завершаемости курса.
GRS = число успешно завершенных курсов / число регистраций на курсы

In [53]:
exam_with_score = df_a.query('assessment_type == "Exam"').id_assessment.unique()
assessments.query('id_assessment in @exam_with_score')

,code_module,code_presentation,id_assessment,assessment_type,date,weight
62,CCC,2014B,24290,Exam,NaN,100.0
72,CCC,2014J,24299,Exam,NaN,100.0
87,DDD,2013B,25340,Exam,240.0,100.0
94,DDD,2013J,25354,Exam,261.0,100.0
101,DDD,2014B,25361,Exam,241.0,100.0
108,DDD,2014J,25368,Exam,NaN,100.0


In [54]:
# Усекли df_a только записями с семестрами, по экзаменам которых ставились оценки

df_RFM = df_a.merge(assessments.query('id_assessment in @exam_with_score')[['code_module', 
                                                                            'code_presentation']], 
                    on=['code_module', 'code_presentation'])
df_RFM

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length
0,CCC,2014B,24286,CMA,18.0,2.0,856294,20,0,20,-51.0,46.0,241
1,CCC,2014B,24286,CMA,18.0,2.0,917586,20,0,90,-29.0,NaN,241
2,CCC,2014B,24286,CMA,18.0,2.0,925553,20,0,60,-232.0,NaN,241
3,CCC,2014B,24286,CMA,18.0,2.0,949548,20,0,70,-95.0,NaN,241
4,CCC,2014B,24286,CMA,18.0,2.0,984619,20,0,80,-179.0,NaN,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49733,DDD,2014J,25368,Exam,NaN,100.0,652680,243,0,91,-144.0,NaN,262
49734,DDD,2014J,25368,Exam,NaN,100.0,652732,249,0,47,-17.0,NaN,262
49735,DDD,2014J,25368,Exam,NaN,100.0,652965,242,0,87,-142.0,NaN,262
49736,DDD,2014J,25368,Exam,NaN,100.0,653051,242,0,27,-60.0,NaN,262


In [55]:
# Для расчета R нам понадобится срок сдачи каждого экзамена. Однако в date есть NaN. При этом срок сдачи экзаменов с 
# заполненной датой совпадает с длительностью семестра, что в принципе логично, тогда заполним NaN датой, равной
# продолжительности семестра

df_RFM \
    .query('assessment_type == "Exam"') \
    .groupby('id_assessment') \
    .agg({'date': ['min', 'max'], 'module_presentation_length': 'max'})

date        module_presentation_length
                 min    max                        max
id_assessment                                         
24290            NaN    NaN                        241
24299            NaN    NaN                        269
25340          240.0  240.0                        240
25354          261.0  261.0                        261
25361          241.0  241.0                        241
25368            NaN    NaN                        262

In [56]:
# Заполним NaN и проверим результат

df_RFM.loc[df_RFM['date'].isnull(), 'date'] = df_RFM.loc[df_RFM['date'].isnull(), 'module_presentation_length']
df_RFM \
    .query('assessment_type == "Exam"') \
    .groupby('id_assessment') \
    .agg({'date': ['min', 'max'], 'module_presentation_length': 'max'})

date        module_presentation_length
                 min    max                        max
id_assessment                                         
24290          241.0  241.0                        241
24299          269.0  269.0                        269
25340          240.0  240.0                        240
25354          261.0  261.0                        261
25361          241.0  241.0                        241
25368          262.0  262.0                        262

In [57]:
df_RFM.date = df_RFM.date.astype('int32')

Принцип формирования сегментов по R
По каждому сочетанию студент+курс+семестр выведем значение R, которое будет скалдываться из среднего по сумме R1 + R2 + R3, где:
R1 = 1 для всех записей (то есть каждый студент точно либо не добрался до экзамена, сдал его после дедлайна или не сдал промежуточные тесты);
R2 = 1 для записей, где время сдачи меньше дедлайна, но больше среднего среди срока сдачи конкретного экзамена (то есть необходимо будет для каждого экзамена вывести средний срок сабмита и с ним сравнивать дату сабмита конктретого студента на конкретном курсе);
R3 = 1 для записей, где средний срок сдачи экзамена ниже среднего по семестру.

Если мы говорим об адаптированном RFM-анализе, то сравнивать средний срок сдачи экзамена студентом в R2 со средним сроком сдачи всех экзаменов всеми студентами не очень корректно, ведь есть более легкие курсы, а есть курсы сложнее, да и срок сдачи в каждом семетре свой, поэтому и сравниваем со средним по семестру.

Приступим к формированию R-сегментов.

Необходимо оставить только записи с экзаменами, но при этом нельзя потерять студентов, не дошедших до экзамена. Через right outer join выберем по ключу курс+семестр+студент студентов, которые не сдавали экзамен, оставим уникальные записи по этому ключу курс+семестр+студент+дата регистрации (ведь на один курс могло быть несколько регистраций).

In [58]:
stud_no_exam = df_RFM \
    .query('assessment_type == "Exam"')[['code_module', 'code_presentation', 'id_student']] \
    .drop_duplicates() \
    .merge(df_RFM, on=['code_module', 'code_presentation', 'id_student'], how="outer", indicator=True) \
    .query('_merge == "right_only"') \
    .drop_duplicates(subset=['code_module', 'code_presentation', 'id_student', 'date_registration']) \
    .drop(columns=['_merge'])

In [59]:
# Заполним этим студентам дату сдачи экзамена и оценку за экзамен, равную -1, чтобы в дальнейшем при 
# расчетах можно  было их не учитывать, отбирая записи >= 0

stud_no_exam.date_submitted = -1
stud_no_exam.score = -1

In [60]:
# Соберем рабочий датафрейм

df_RFM = pd.concat([df_RFM.query('assessment_type == "Exam"'), stud_no_exam])

In [61]:
# Добавим флаг возможности рассмотрения оценки экзамена (сданы все промежуточные тесты)

df_RFM = df_RFM \
    .merge(admitted_students, on=['id_student', 'code_module', 'code_presentation'], how='left')

In [62]:
# Пропущенное значение означает, что в указанном семестре курса оценка студента не может быть принята, так как
# не сданы все промежуточные тесты

df_RFM['admitted'] = df_RFM['admitted'].fillna(False)

In [63]:
# Посчитаем средний срок сдачи экзамена студентами по каждому экзамену, добавим в рабочий датафрейм и рассчитаем R

df_RFM = df_RFM \
    .merge(df_RFM
           .query('date_submitted >= 0')
           .groupby(['code_module', 'code_presentation'], as_index=False)
           .agg({'date_submitted': 'mean'})
           .rename(columns={'date_submitted': 'mean_submitted'}),
           on=['code_module', 'code_presentation'],
           how='left') \
    .assign(R1=True, 
            R2=lambda x: (x.date_submitted <= x.module_presentation_length) & df_RFM.admitted, 
            R3=lambda x: (x.date_submitted <= x.mean_submitted) & df_RFM.admitted,
            R=lambda x: sum([x.R1, x.R2, x.R3]))
df_RFM.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,module_presentation_length,admitted,mean_submitted,R1,R2,R3,R
0,CCC,2014B,24290,Exam,241,100.0,558914,230,0,32,-74.0,NaN,241,True,231.617135,True,True,True,3
1,CCC,2014B,24290,Exam,241,100.0,559706,234,0,78,-22.0,NaN,241,True,231.617135,True,True,False,2
2,CCC,2014B,24290,Exam,241,100.0,559770,230,0,54,-22.0,NaN,241,True,231.617135,True,True,True,3
3,CCC,2014B,24290,Exam,241,100.0,560114,230,0,64,-281.0,NaN,241,True,231.617135,True,True,True,3
4,CCC,2014B,24290,Exam,241,100.0,560311,234,0,100,-28.0,NaN,241,True,231.617135,True,True,False,2


In [64]:
# Пограничных значений немного (1.5 округляется до 2, 2.5 округлится до 3). При увеличении этих чисел будет 
# иметь смысл увеличить число сегментов.

df_RFM.groupby('id_student').agg({'R': 'mean'}).value_counts()

R       
1.000000    4148
3.000000    1704
2.000000    1431
1.500000     117
2.500000      78
1.666667       6
1.333333       5
2.333333       1
dtype: int64

In [65]:
# Начнем собирать итоговый датафрейм

segmented_RFM = df_RFM \
    .groupby('id_student', as_index=False) \
    .agg({'R': 'mean'}) \
    .round()

In [66]:
segmented_RFM['R'] = segmented_RFM['R'].astype('int32')

Начинаем считать F
1. Посчитаем число экзаменов, сдававшихся студентом по каждому предмету и семестру.
2. Если оценка за экзамен меньше 40 и студент не допущен к экзамену, то этот экзамен не идет в расчет числа успешно сданных
3. Посчитаем число регистраций студента на все курсы
4. Вычислим GRS
5. Вычислим сегмент для каждого студента: F = F1 + F2 + F3, где:
F1=1, если GRS >= 0 (сдано 0 или больше экзаменов)
F2=1, если GRS > 0 (сдан хотя бы один эказмен)
F3=1, если GRS == 1 (сданы все экзамены, с курсов, на которые студент регистрировался)

In [67]:
# Начнем считать число сданных экзаменов exam_count

stud_exams = df_RFM \
    .query('assessment_type == "Exam"') \
    .groupby(['code_module', 'code_presentation', 'id_student'], as_index=False) \
    .agg({'id_assessment': 'count'}) \
    .rename(columns={'id_assessment': 'exam_count'})
stud_exams.head()

,code_module,code_presentation,id_student,exam_count
0,CCC,2014B,29764,1
1,CCC,2014B,29820,1
2,CCC,2014B,40604,1
3,CCC,2014B,42638,1
4,CCC,2014B,46605,1


Добавим результаты о числе сданных экзаменов, объеденим через left и заполним NaN нулями (таким образом число сданных экзаменов у студентов, зарегистрировавшихся на курс, но не дошедших до экзамена будет равно 0)

In [68]:
df_RFM = df_RFM \
    .merge(stud_exams, on=['code_module', 'code_presentation', 'id_student'], how='left')

In [69]:
df_RFM['exam_count'] = df_RFM['exam_count'].fillna(0).astype('int32')

In [70]:
# Проверим, что все верно и каждый студент в рамках одного семестра имеет только 1 сданный экзамен

df_RFM.exam_count.unique()

array([1, 0], dtype=int32)

In [71]:
# Экзамен сдан, если оценка не ниже 40 и студент допущен к экзамену

df_RFM['exam_count'] = df_RFM['exam_count'] * (df_RFM['score'] >= 40) * df_RFM['admitted']

In [72]:
# Посчитаем число регистраций на курсы каждого студента и присоединим к рабочему датафрейму

reg_by_student = studentRegistration \
    .groupby('id_student', as_index=False) \
    .agg({'date_registration': 'count'}) \
    .rename(columns={'date_registration': 'registration_count'}) \
    .query('registration_count > 0')
reg_by_student.head()

,id_student,registration_count
0,3733,1
1,6516,1
2,8462,2
3,11391,1
4,23629,1


In [73]:
df_RFM = df_RFM \
    .merge(reg_by_student, on='id_student', how='left')

In [74]:
# Так как число регистраций будет в знаметеле, то необходимо строго проверить на пропущенные значения и 0

df_RFM.query('registration_count == 0')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,...,date_unregistration,module_presentation_length,admitted,mean_submitted,R1,R2,R3,R,exam_count,registration_count


In [75]:
df_RFM[df_RFM.registration_count.isna()]

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,...,date_unregistration,module_presentation_length,admitted,mean_submitted,R1,R2,R3,R,exam_count,registration_count


In [76]:
# Теперь можем посчитать GRS

df_RFM['GRS'] = df_RFM['exam_count'] / df_RFM['registration_count']
df_RFM

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,...,module_presentation_length,admitted,mean_submitted,R1,R2,R3,R,exam_count,registration_count,GRS
0,CCC,2014B,24290,Exam,241,100.0,558914,230,0,32,...,241,True,231.617135,True,True,True,3,0,2,0.0
1,CCC,2014B,24290,Exam,241,100.0,559706,234,0,78,...,241,True,231.617135,True,True,False,2,1,2,0.5
2,CCC,2014B,24290,Exam,241,100.0,559770,230,0,54,...,241,True,231.617135,True,True,True,3,1,2,0.5
3,CCC,2014B,24290,Exam,241,100.0,560114,230,0,64,...,241,True,231.617135,True,True,True,3,1,2,0.5
4,CCC,2014B,24290,Exam,241,100.0,560311,234,0,100,...,241,True,231.617135,True,True,False,2,1,2,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8344,DDD,2014J,25363,TMA,41,10.0,634693,-1,0,-1,...,262,False,242.791579,True,False,False,1,0,1,0.0
8345,DDD,2014J,25363,TMA,41,10.0,655972,-1,0,-1,...,262,False,242.791579,True,False,False,1,0,1,0.0
8346,DDD,2014J,25363,TMA,41,10.0,677444,-1,0,-1,...,262,False,242.791579,True,False,False,1,0,1,0.0
8347,DDD,2014J,25363,TMA,41,10.0,233868,-1,0,-1,...,262,False,242.791579,True,False,False,1,0,2,0.0


In [77]:
# Рассчитаем F, F1, F2, F3
# Группируем по студенту и устанавливаем значения F
# F1 можем автоматически установаить в True

df_RFM_F = df_RFM \
    .groupby('id_student', as_index=False) \
    .agg({'GRS': 'mean'}) \
    .assign(F1=True, 
            F2=lambda x: x.GRS > 0, 
            F3=lambda x: x.GRS == 1,
            F=lambda x: sum([x.F1, x.F2, x.F3]).astype('int32'))[['id_student', 'F']]
df_RFM_F.head()

,id_student,F
0,8462,1
1,23698,3
2,24213,2
3,25261,1
4,25572,1


Рассчитаем M
1. Сгруппируем рабочий датафрейм по студенту, отобрав оценки не ниже 0, агрегируем среднюю оценку
2. Установим значение M = 1 если средняя оценка ниже 40, М = 2 если в диапазоне [40, 75) и М = 3, если в диапазоне [75, 100]
3. Присоединим слева эти данные к финальному датафрейму, а пропущенные значения заполним 1, так как мы отсеили
оценки ниже 0, а они относят студента к 1 сегменту M.

In [78]:
df_RFM_M = df_RFM \
    .query('score >= 0') \
    .groupby('id_student', as_index=False) \
    .agg({'score': 'mean'}) \
    .assign(M1=True, 
            M2=lambda x: x.score >= 40, 
            M3=lambda x: x.score > 75,
            M=lambda x: sum([x.M1, x.M2, x.M3]))[['id_student', 'M']]
df_RFM_M.head()

,id_student,M
0,23698,3
1,24213,2
2,27116,3
3,28046,2
4,28787,2


In [79]:
segmented_RFM = segmented_RFM \
    .merge(df_RFM_F, on='id_student', how='left') \
    .merge(df_RFM_M, on='id_student', how='left')
segmented_RFM.head()

,id_student,R,F,M
0,8462,1,1,NaN
1,23698,3,3,3.0
2,24213,2,2,2.0
3,25261,1,1,NaN
4,25572,1,1,NaN


In [80]:
# Ожидаемо NA есть только в М сегментах, заполним их 1

segmented_RFM.isna().sum()

id_student       0
R                0
F                0
M             2857
dtype: int64

In [81]:
segmented_RFM['M'] = segmented_RFM['M'].fillna(1).astype('int32')

In [82]:
segmented_RFM

,id_student,R,F,M
0,8462,1,1,1
1,23698,3,3,3
2,24213,2,2,2
3,25261,1,1,1
4,25572,1,1,1
...,...,...,...,...
7485,2695608,3,3,2
7486,2696376,1,1,1
7487,2697181,1,1,3
7488,2698251,3,3,2


In [83]:
segmented_RFM.R.value_counts()

1    4153
3    1704
2    1633
Name: R, dtype: int64

In [84]:
segmented_RFM.F.value_counts()

1    4516
3    2160
2     814
Name: F, dtype: int64

In [85]:
segmented_RFM.M.value_counts()

1    3400
2    2456
3    1634
Name: M, dtype: int64